In [15]:
from get_data_from_XML import *
import pydicom as dicomio
from utils import *
from get_data_from_XML import get_category
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image, ImageDraw



In [16]:
annotation_path = '/Users/khanhhuyen/Documents/IAI/lung_CT/Annotation/A0002'
num_classes = 4
class_list = get_category('category.txt')
dicom_mode = 'CT'
dicom_path = '/Users/khanhhuyen/Documents/IAI/lung_CT/Lung-PET-CT-Dx/manifest-1675442780019/Lung-PET-CT-Dx/Lung_Dx-A0002'

In [17]:
def loadFileInformation(filename):
    information = {}
    ds = dicomio.read_file(filename, force=True)
    information['dicom_num'] = ds.SOPInstanceUID
    return information

In [18]:
def getUID_path(path):
    dict = {}
    list = os.listdir(path)

    for date in list:
        date_path = os.path.join(path, date)
        series_list = os.listdir(date_path)
        series_list.sort()

        for series in series_list:
            series_path = os.path.join(date_path, series)
            dicom_list = os.listdir(series_path)
            dicom_list.sort()

            for dicom in dicom_list:
                dicom_path = os.path.join(series_path, dicom)
                info = loadFileInformation(dicom_path)
                dict[info['dicom_num']] = (dicom_path, dicom)

    return dict
dictionary = getUID_path(dicom_path)

AttributeError: 'FileDataset' object has no attribute 'SOPInstanceUID'

In [ ]:
def showImage(img, title='image', t=0, esc=False):
    cv2.imshow(title, img)
    if esc:
        while cv2.waitKey(0) != 27:
            if cv2.getWindowProperty(title, cv2.WND_PROP_VISIBLE)<=0:
                break
    else:
        cv2.waitKey(t)
    cv2.destroyWindow(title)

def class_colors(num_colors):
    class_colors = []
    for i in range(0, num_colors):
        hue = 255 * i / num_colors
        col = np.zeros((1, 1, 3)).astype("uint8")
        col[0][0][0] = hue
        col[0][0][1] = 128  # Saturation
        col[0][0][2] = 255  # Value
        cvcol = cv2.cvtColor(col, cv2.COLOR_HSV2BGR)
        col = (int(cvcol[0][0][0]), int(cvcol[0][0][1]), int(cvcol[0][0][2]))
        class_colors.append(col)

    return class_colors

def roi2rect(img_name, img_np, img_data, label_list):
    colors = class_colors(len(label_list))
    for rect in img_data:
        bounding_box = [rect[0], rect[1], rect[2], rect[3]]
        # print(bounding_box)
        xmin = int(bounding_box[0])
        ymin = int(bounding_box[1])
        xmax = int(bounding_box[2])
        ymax = int(bounding_box[3])
        pmin = (xmin, ymin)
        pmax = (xmax, ymax)

        label_array = rect[4:]
        # print(label_array)
        #find the first index of 1
        index = int(np.where(label_array == 1)[0])
        # index = int(np.where(label_array == np.float(1))[0])
        label = label_list[index]

        color = tuple(map(int, np.uint8(np.random.uniform(0, 255, 3))))
        color = colors[index]
        cv2.rectangle(img_np, pmin, pmax, color, 2)

        text_top = (xmin, ymin - 10)
        text_bot = (xmin + 80, ymin + 5)
        text_pos = (xmin + 5, ymin)
        cv2.rectangle(img_np, text_top, text_bot, colors[index], -1)
        cv2.putText(img_np, label, text_pos, cv2.FONT_HERSHEY_PLAIN, cv2.FONT_HERSHEY_PLAIN, 1, 1, 2)
    
    return img_np



In [ ]:
dcm_names = []
frames = []
if os.path.isdir(annotation_path):
    annotations = XML_preprocessor(annotation_path, num_classes=num_classes).data
    for k, v in (pbar:=tqdm(annotations.items())):
        # # dcm_name = k + '.dcm'
        dcm_path, dcm_name = dictionary[k[:-4]]
        image_data = v

        if dicom_mode == 'CT':
            matrix, frame_num, width, height, ch = loadFile(os.path.join(dcm_path))
            img_bitmap = MatrixToImage(matrix[0], ch)
        elif dicom_mode == 'PET':
            img_array, frame_num, width, height, ch = loadFile(dcm_path)
            img_bitmap = PETToImage(img_array, color_reversed=True)
        # print(dcm_name, img_bitmap, image_data, class_list)
        pbar.set_description(f'Processing {dcm_name}')
        result = roi2rect(img_name=dcm_name, img_np=img_bitmap, img_data=image_data, label_list=class_list)
        
        # save to parent folder of dcm_path
        
        save_path = os.path.join(os.path.dirname(os.path.dirname(dcm_path)), 'result')
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        save_name = os.path.join(save_path, dcm_name.split('.')[0] + '.png')
        
        
        img = Image.fromarray(result)
        frames.append(img)
        img.save(save_name)
        dcm_names.append(dcm_name)

        
        

        # break

# sort dcm_names
dcm_names.sort()
print(dcm_names) 
# save to gif
frames[0].save(os.path.join(save_path, 'result.gif'), format='GIF', append_images=frames[1:], save_all=True, duration=100, loop=0)

Processing 1-14.dcm: 100%|██████████| 18/18 [00:00<00:00, 27.44it/s]


['1-09.dcm', '1-10.dcm', '1-11.dcm', '1-11.dcm', '1-12.dcm', '1-12.dcm', '1-13.dcm', '1-13.dcm', '1-14.dcm', '1-14.dcm', '1-15.dcm', '1-15.dcm', '1-16.dcm', '1-16.dcm', '1-17.dcm', '1-17.dcm', '1-18.dcm', '1-18.dcm']
